In [ ]:
# Attention Mask : 실제토큰 1 / 0 패딩
# Token Type IDS(Segment IDs) : 두 개의 문장(A / B) 구성될때 각 토큰이 어느 문장에 속하는지 알려주는 임베딩
# CLS Token Pooling : [CLS] + token + [SEP]

In [ ]:
# 1. Bert Tokenizer : 단어를 의미있는 조각(subword)로 나눕니다 unbelievable "un" 'believ" "able"
from transformers import BertTokenizer